# Start the Shapash Web App on a sample dataset

<b>With this tutorial you:</b><br />
Understand how shapash works with a simple use case<br />
Start WebApp to understand your model and save these results

Contents:
- Build a Regressor
- Compile Shapash SmartExplainer
- Start Shapash WebApp
- Export synt with to_pandas function
- Save Shapash object in pickle file

Data from Kaggle [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [1]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

## Building Supervized Model 

In [2]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

In [4]:
house_df.shape

(1460, 73)

In [5]:
house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,...,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,...,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Northridge,...,0,0,0,0,0,12,2008,Warranty Deed - Conventional,Normal Sale,250000


#### Encoding Categorical Features 

In [6]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/category_encoders/utils.py:28: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/category_encoders/utils.py:50: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/category_encoders/utils.py:50: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/category_encoders/utils.py:50: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/Users/valencar/anaconda3/envs/py310/lib/python3

#### Train / Test Split

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

#### Model Fitting

In [8]:
regressor = LGBMRegressor(n_estimators=100).fit(Xtrain,ytrain)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007188 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2986
[LightGBM] [Info] Number of data points in the train set: 1095, number of used features: 66
[LightGBM] [Info] Start training from score 182319.757078


## Understanding my model with shapash

#### Declare and Compile SmartExplainer 

In [9]:
from shapash import SmartExplainer

In [10]:
xpl = SmartExplainer(
    model=regressor,
    preprocessing=encoder,   # Optional: compile step can use inverse_transform method
    features_dict=house_dict # optional parameter, specifies label for features name 
)

In [11]:
xpl.compile(x=Xtest,
            y_target=ytest # Optional: allows to display True Values vs Predicted Values
           )

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/utils/check.py:138: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



#### Start WebApp

In [17]:
app = xpl.run_app(title_story='House Prices', port=8020)

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/webapp/smart_app.py:126: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/webapp/utils/utils.py:41: FutureWarning:

Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead

INFO:root:Your Shapash application run on http://Vladimirs-MacBook-Pro.local:8020/
INFO:root:Use the method .kill() to down your app.


/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/explainer/smart_plotter.py:3191: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/explainer/smart_plotter.py:3208: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/explainer/smart_plotter.py:1267: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/explainer/smart_plotter.py:3191: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame

Link to App: <br />
[shapash-monitor link](https://shapash-demo.ossbymaif.fr/)

#### Stop the WebApp after using it

In [13]:
app.kill()

#### Export local explaination in DataFrame

In [14]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    threshold=5000,
)

/Users/valencar/anaconda3/envs/py310/lib/python3.10/site-packages/shapash/manipulation/summarize.py:92: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [15]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
259,210110.151269,Ground living area square feet,1792,13901.94265,Overall material and finish of the house,7,13388.78542,Total square feet of basement area,963,-5785.099112
268,177235.867901,Ground living area square feet,2192,27473.618384,Overall material and finish of the house,5,-26452.634884,Overall condition of the house,8,6804.442167
289,112646.087826,Overall material and finish of the house,5,-25938.289451,Ground living area square feet,900,-16286.662175,Total square feet of basement area,882,-5586.17177
650,74998.294879,Overall material and finish of the house,4,-34883.385439,Ground living area square feet,630,-21321.74224,Total square feet of basement area,630,-12563.167906
1234,139796.103166,Overall material and finish of the house,5,-26786.433337,Ground living area square feet,1188,-11315.446699,NaN,NaN,NaN


#### Save SmartExplainer in Pickle File

You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again

In [16]:
xpl.save('./xpl.pkl')